In [1]:
import random
import math
import os
import json

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

In [4]:
def calculate_rotated_bbox_for_yolo_v8(data: dict, normalize: bool = False) -> list[tuple]:
    """
    Calculate the absolute coordinates of the corners of a rotated bounding box or 
    normalize them if required.
    
    This function takes the bounding box information and applies rotation to get the 
    coordinates of the four corners after rotation. If normalization is required, it 
    scales these coordinates to be between 0 and 1.

    Args:
    - data (dict): A dictionary containing the bounding box information.
    - normalize (bool): A flag to indicate if the coordinates should be normalized.

    Returns:
    - list of tuples: A list containing tuples, each with the (x, y) coordinates 
                        of a corner of the bounding box, either absolute or normalized.
    """
    # Convert percentages to absolute values if not normalizing
    if not normalize:
        width = data['width'] / 100 * data['original_width']
        height = data['height'] / 100 * data['original_height']
        top_left_x = data['x'] / 100 * data['original_width']
        top_left_y = data['y'] / 100 * data['original_height']
    else:
        # If normalizing, use percentages as they are
        width = data['width']
        height = data['height']
        top_left_x = data['x']
        top_left_y = data['y']
    
    # Convert rotation angle to radians and adjust for clockwise rotation
    angle_rad = math.radians(data['rotation'])

    # Coordinates of the bounding box corners prior to rotation
    corners = [(0, 0), (width, 0), (width, height), (0, height)]

    # Rotate each corner around the top-left corner
    rotated_corners = []
    for x, y in corners:
        # Apply the rotation matrix to each corner point
        rotated_x = x * math.cos(angle_rad) - y * math.sin(angle_rad)
        rotated_y = x * math.sin(angle_rad) + y * math.cos(angle_rad)
        
        # Translate the rotated points back by adding the coordinates of the top-left corner
        # If normalizing, convert the coordinates to a scale from 0 to 1
        if normalize:
            rotated_corners.append(((rotated_x + top_left_x) / data['original_width'], 
                                    (rotated_y + top_left_y) / data['original_height']))
        else:
            rotated_corners.append((rotated_x + top_left_x, rotated_y + top_left_y))

    return rotated_corners

def convert_annotations_to_yolo_obb(json_folder_path: str, output_folder_path: str, 
                                    class_list: list, normalize: bool = False):
    """
    Convert rotated bounding box annotations from JSON files to YOLO OBB format and save to TXT files.
    Optionally normalize the bounding box coordinates.

    Args:
    - json_folder_path (str): The path to the folder containing JSON annotation files.
    - output_folder_path (str): The path to the folder where TXT files will be saved.
    - class_list (list): A list of class names ordered according to their class index.
    - normalize (bool): A flag to indicate if the coordinates should be normalized.

    Outputs:
    - TXT files containing the annotations in YOLO OBB format, saved to the destination folder.
    """
    # Create the destination folder if it does not exist
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)
    
    # Loop through all the files in the json directory
    for file_name in os.listdir(json_folder_path):
        if file_name.endswith('.json'):
            # Read the JSON file
            with open(os.path.join(json_folder_path, file_name), 'r') as json_file:
                data = json.load(json_file)
            
            # Prepare the content for the TXT file
            txt_content = []
            for annotation in data['label']:
                # Calculate the rotated bounding box coordinates
                corners = calculate_rotated_bbox_for_yolo_v8(annotation, normalize=normalize)
                # Get the class index
                class_index = class_list.index(annotation['rectanglelabels'][0])
                # Convert coordinates to the YOLO OBB format, absolute or normalized
                yolo_obb = [class_index] + [val for corner in corners for val in corner]
                txt_content.append(' '.join(map(str, yolo_obb)))
            
            # Write the content to the corresponding TXT file
            txt_file_name = os.path.splitext(data['image'].split('/')[-1])[0] + '.txt'
            with open(os.path.join(output_folder_path, txt_file_name), 'w') as txt_file:
                txt_file.write('\n'.join(txt_content))

In [9]:
source_path = "/Users/jocareher/Downloads/face_dataset/test/images"
output_path = "/Users/jocareher/Downloads/face_dataset/test/labels"
normalize = True
class_list = ["3/4_left_sideview", "3/4_rigth_sideview", "Frontal", "Left_sideview", "Right_sideview"]

convert_annotations_to_yolo_obb(json_folder_path=source_path,
                                output_folder_path=output_path,
                                class_list=class_list,
                                normalize=normalize)

In [ ]:
import os
import random
from PIL import Image

def create_yolov8_pairs(root_directory: str) -> list[tuple]:
    """
    Traverse the 'images' and 'labels' subdirectories within the specified root directory.
    Pair each image with its corresponding annotation file, if available.

    Args:
    - root_directory (str): The path to the root directory containing 'images' and 'labels' folders.

    Returns:
    - List of tuples: Each tuple contains the path to an image file and a list of annotation strings.
    """
    # Path to the subdirectory containing image files
    images_dir = os.path.join(root_directory, 'images')
    # Path to the subdirectory containing annotation files
    labels_dir = os.path.join(root_directory, 'labels')

    pairs = []
    # Loop through all files in the images directory
    for image_name in os.listdir(images_dir):
        # Check if the file is a JPEG image
        if image_name.endswith('.jpg'):
            # Extract the base name without the file extension
            base_name = os.path.splitext(image_name)[0]
            # Construct the corresponding label file name
            label_name = base_name + '.txt'
            # Full paths to the image and label files
            image_path = os.path.join(images_dir, image_name)
            label_path = os.path.join(labels_dir, label_name)
            
            # Check if the annotation file exists
            if os.path.exists(label_path):
                # Read all lines from the annotation file
                with open(label_path, 'r') as file:
                    annotations = file.readlines()
                # Append the image path and its annotations as a tuple to the pairs list
                pairs.append((image_path, annotations))
    return pairs

def draw_yolov8_annotations_on_images(pairs: list[tuple],
                               class_list: list[str],
                               num_images_to_display: int,
                               show_labels: bool = True,
                               show_axis: str = "on") -> None:
    """
    Draw annotations on images as polygons and display them in a grid.
    Optionally include class labels aligned with the top edge of the bounding box.

    Args:
    - pairs (list of tuples): A list containing tuples of image paths and annotation data.
    - class_list (list of str): A list of class names ordered by their corresponding class index.
    - num_images_to_display (int): The number of images to display on the grid.
    - show_labels (bool, optional): If True, display class labels. Default is True.
    - show_axis (str, optional): Control the visibility of the axis. Default is "on".

    Outputs:
    - Displays a grid of images with the respective annotations.
    """
    # Select a random subset of image-annotation pairs
    selected_pairs = random.sample(pairs, min(num_images_to_display, len(pairs)))
    
    # Determine the number of rows and columns for the grid based on the number of images
    grid_cols = int(np.ceil(np.sqrt(num_images_to_display)))
    grid_rows = int(np.ceil(num_images_to_display / grid_cols))
    # Create a grid of subplots
    fig, axs = plt.subplots(nrows=grid_rows, ncols=grid_cols, figsize=(15, 15))
    axs = axs.flatten()  # Flatten to 1D array for easy indexing

    # Loop through the axes and hide any that won't be used
    for ax in axs[num_images_to_display:]:
        ax.axis('off')

    for idx, ax in enumerate(axs[:num_images_to_display]):
        # Extract the image path and annotations for the current index from the selected pairs
        image_path, annotation_data = selected_pairs[idx]
        # Open the image file and display it on the current axis
        img = Image.open(image_path)
        ax.imshow(img)
        
        # Iterate over each annotation for the current image
        for annotation in annotation_data:
            # Extract the class index and convert it to the class name
            class_index = int(annotation.split(' ')[0])
            class_name = class_list[class_index]
            # Parse the annotation coordinates and reshape them into a 2x4 matrix
            points = list(map(float, annotation.strip().split(' ')[1:]))
            points = np.array(points).reshape((4, 2))
            
            # Create a polygon patch from the annotation points and add it to the axis
            poly = patches.Polygon(points, closed=True, fill=False, edgecolor='blue')
            ax.add_patch(poly)
            
            # If labels should be shown, calculate the text properties and display it
            if show_labels:
                top_edge_vec = points[1] - points[0]  # Vector representing the top edge of the box
                angle = np.arctan2(top_edge_vec[1], top_edge_vec[0])
                
                # Set the position for the label text at the midpoint of the top edge
                label_pos = (points[0] + points[1]) / 2
                text_x, text_y = label_pos
                margin = 3  # Margin for the text position above the top edge
                
                # Adjust text position based on the orientation of the top edge
                if top_edge_vec[0] < 0:  # If the edge is oriented to the left
                    angle -= np.pi  # Adjust the angle to keep text orientation consistent

                # The text is placed above the top edge, considering the margin
                ax.text(text_x, text_y - margin, class_name, rotation=np.degrees(angle),
                        color='red', fontsize=9, ha='center', va='bottom', rotation_mode='anchor')
                
        # Display axis on the images
        ax.axis(show_axis)
    
    plt.tight_layout()
    plt.show()

In [2]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)

# Train the model with 2 GPUs
results = model.train(data="/Users/jocareher/Library/CloudStorage/OneDrive-Personal/Educación/PhD_UPF_2023/Face_Detection/configs/yolo.yaml", epochs=100, imgsz=640, device='mps')

100%|██████████| 6.23M/6.23M [00:00<00:00, 20.5MB/s]


Ultralytics YOLOv8.1.9 🚀 Python-3.9.18 torch-2.2.0 MPS (Apple M2 Max)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/Users/jocareher/Library/CloudStorage/OneDrive-Personal/Educación/PhD_UPF_2023/Face_Detection/configs/yolo.yaml, epochs=100, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=mps, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_

train: Scanning /Users/jocareher/Downloads/face_dataset/train/labels... 5342 images, 0 backgrounds, 19 corrupt: 100%|██████████| 5342/5342 [00:01<00:00, 3748.33it/s]

train: WARNING ⚠️ /Users/jocareher/Downloads/face_dataset/train/images/face_img_3134.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0053]
train: WARNING ⚠️ /Users/jocareher/Downloads/face_dataset/train/images/face_img_3144.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0022]
train: WARNING ⚠️ /Users/jocareher/Downloads/face_dataset/train/images/face_img_3281.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.055]
train: WARNING ⚠️ /Users/jocareher/Downloads/face_dataset/train/images/face_img_3346.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0361]
train: WARNING ⚠️ /Users/jocareher/Downloads/face_dataset/train/images/face_img_3366.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0255]
train: WARNING ⚠️ /Users/jocareher/Downloads/face_dataset/train/images/face_img_6001.jpg: ignoring corrupt ima


val: Scanning /Users/jocareher/Downloads/face_dataset/val/labels... 798 images, 0 backgrounds, 6 corrupt: 100%|██████████| 798/798 [00:00<00:00, 3912.54it/s]

...

WARNING ⚠️ NMS time limit 3.600s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  28%|██▊       | 7/25 [01:33<04:00, 13.37s/it]

WARNING ⚠️ NMS time limit 3.600s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  32%|███▏      | 8/25 [01:45<03:45, 13.25s/it]


KeyboardInterrupt: 